In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

## Homework 2, part 1

We build a CNN to classify the galaxies.

Goal: 75% or more!

In [ ]:
!wget https://www.dropbox.com/s/apl6g5g9svhnfyg/Dataset.zip

In [ ]:
!unzip Dataset.zip

In [ ]:
path_to_training_data = 'Dataset/train/'
path_to_validation_data = 'Dataset/validation/'

In [ ]:
%load_ext autoreload
%autoreload 2

### Dataloader

Reuse the same as HW1, but make sure that the shape is (3, 69, 69). This because pretrained models have been trained on color images that have three channels (R,G,B). We need our input to match thath, at least in it's shape.

In [ ]:
# Example of torch repeating an array

some_tensor = torch.rand(1,69,69)
print(some_tensor.shape)
some_tensor = some_tensor.repeat(3,1,1)
print(some_tensor.shape)

In [ ]:
from dataloader import CustomDataset

In [ ]:
training_ds = CustomDataset(path_to_training_data,transform=True)
validation_ds = CustomDataset(path_to_validation_data)

some_random_idx = 2
x,y = training_ds[some_random_idx]
x.shape

In [ ]:
training_dataloader = DataLoader(training_ds,batch_size = 300,shuffle=True)
valid_dataloader = DataLoader(validation_ds,batch_size = 300)

In [ ]:
#make sure that you are getting the right dimensions - (Batch size, 3, 69,69)

for x,y in training_dataloader:
    print(x.shape,y.shape)
    break

### Download the pre-trained model

From the list https://pytorch.org/vision/stable/models.html, get any pretrained models you want.

I show you vgg11 for example (but you can use alexnet or resnet also).

In [ ]:
import torchvision.models as models

pretrained_model = models.vgg11(pretrained=True)

In [ ]:
# Look at the internal structure..

# features (use a Sequential model) -- try to understand what each layer does
# classifier (returns an array of 1000, it's a simply FC network)
# avgpool allows you to use this with any image size, simply rescale the input to a preffered size

pretrained_model

In [ ]:
# the pretrained models have an output shape that matches the number of classes they were trained on

pretrained_model(x).shape

In [ ]:
# output size of the features layer.. after flattening it, this will be the input for the classifier
pretrained_model.features(x).shape

### The model

You have to build a model that has the same feature structure, but a different classifier. In fact out output should be size 10, since we have only 10 classes.

In [ ]:
from model import Net

In [ ]:
net = Net()
print(net)

In [ ]:
# You should get (batch size, 10)
net(x).shape

Now, we copy the feature weights from the pretrained model.

If you change one of the layer in the feature part, this command won't work (try!).

In [ ]:
# state_dict is a dictionary containing every weight in every layer
# we want to copy only the feature part

pretrained_dict = pretrained_model.state_dict()
state_dict = net.state_dict()

for key in state_dict.keys():
    if 'features' not in key:
        continue
    if key in pretrained_dict.keys():
        state_dict[key] = pretrained_dict[key]

net.load_state_dict(state_dict)

## Training and validation

Same code as homework 1!

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001) 

In [ ]:
def compute_accuracy_and_loss(dataloader,net):
    total = 0
    correct = 0
    
    loss = 0
    
    if torch.cuda.is_available():
        net.cuda()
    net.eval()
    
    n_batches = 0
    with torch.no_grad():
        for x,y in dataloader:
            n_batches+=1
            
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            pred = net(x)
            
            loss+= loss_func(pred,y).item()
            
            pred = torch.argmax(pred,dim=1)

            correct+=len(torch.where(pred==y)[0])
            total+=len(y)
    loss = loss/n_batches      
    return correct/total, loss

In [ ]:
if torch.cuda.is_available():
    net.cuda()

In [ ]:
n_epochs = 50

training_loss_vs_epoch = []
validation_loss_vs_epoch = []

training_acc_vs_epoch = []
validation_acc_vs_epoch = []

pbar = tqdm( range(n_epochs) )

for epoch in pbar:
    
    if len(validation_loss_vs_epoch) > 1:
        print('epoch',epoch,' val acc:'+'{0:.5f}'.format(validation_acc_vs_epoch[-1])+
              ', train acc:'+'{0:.5f}'.format(training_acc_vs_epoch[-1]))
    
    net.train() # put the net into "training mode"
    
    for x,y in training_dataloader:
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
            
        optimizer.zero_grad()
        pred = net(x)
        loss = loss_func(pred,y)
        loss.backward()
        optimizer.step()
    
    net.eval() #put the net into evaluation mode
    train_acc, train_loss = compute_accuracy_and_loss(training_dataloader,net)
    valid_acc, valid_loss =  compute_accuracy_and_loss(valid_dataloader,net)
         
    training_loss_vs_epoch.append(train_loss)    
    training_acc_vs_epoch.append( train_acc )
    
    validation_acc_vs_epoch.append(valid_acc)
    
    validation_loss_vs_epoch.append(valid_loss)
    if len(validation_loss_vs_epoch)==1 or validation_loss_vs_epoch[-2] > validation_loss_vs_epoch[-1]:
        torch.save(net.state_dict(), 'trained_model.pt')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,3))

ax[0].plot(training_loss_vs_epoch,label='training')
ax[0].plot(validation_loss_vs_epoch,label='validation')

ax[1].plot(training_acc_vs_epoch)
ax[1].plot(validation_acc_vs_epoch)

ax[0].set_ylabel('loss')
ax[1].set_ylabel('accuracy')
for i in range(2):
    ax[i].set_xlabel('epoch')
plt.show()